In [ ]:
import pandas as pd
import requests
import geopandas as gpd

In [19]:
#create a df starting in 2013 and drop a bunch of columns that aren't need right now
url ='https://educationdata.urban.org/api/v1/schools/ccd/directory/2013/?state_location=PA'
r = requests.get(url)
json = r.json()
df = pd.DataFrame(json['results'])
new_df = df.sort_values(by=['enrollment'],ascending=False)
new_df.drop(columns=['ncessch','leaid','state_leaid','seasch','high_cedp','middle_cedp','ungrade_cedp','state_leg_district_lower','state_leg_district_upper','ncessch_num','congress_district_id','direct_certification','lunch_program','magnet','shared_time','virtual','teachers_fte','free_lunch','reduced_price_lunch','free_or_reduced_price_lunch','elem_cedp','bureau_indian_education','title_i_status','title_i_eligible','title_i_schoolwide','charter','highest_grade_offered','lowest_grade_offered','school_status','school_type','school_level','urban_centric_locale','cbsa','csa','phone'],inplace=True)

In [20]:
#function to loop through and add a new column for enrollment by year
def add_years():
    for i in range(14,21):
        global new_df
        url ='https://educationdata.urban.org/api/v1/schools/ccd/directory/' + '20'+ str(i) + '/?state_location=PA'
        r = requests.get(url)
        json = r.json()
        df = pd.DataFrame(json['results'])
        new = df.filter(['school_id','enrollment'], axis=1)
        new.rename(columns = {'enrollment':'enrollment'+str(i)}, inplace = True)
        new_df = new_df.merge(new, on='school_id', how='left')
    return new_df
        
add_years()

,year,school_id,school_name,lea_name,street_mailing,city_mailing,state_mailing,zip_mailing,street_location,city_location,...,longitude,county_code,enrollment,enrollment14,enrollment15,enrollment16,enrollment17,enrollment18,enrollment19,enrollment20
0,2013,00534,PENNSYLVANIA CYBER CS,PENNSYLVANIA CYBER CS,652 MIDLAND AVE,MIDLAND,PA,15059,652 MIDLAND AVE,MIDLAND,...,-80.4549,42007,10389,9344.0,9058.0,9654.0,9664.0,10100.0,9840.0,NaN
1,2013,00896,AGORA CYBER CS,AGORA CYBER CS,995 OLD EAGLE SCH RD SUITE 315,WAYNE,PA,19087,995 OLD EAGLE SCH RD,WAYNE,...,-75.4103,42029,9490,9290.0,8153.0,5867.0,6022.0,5499.0,5135.0,NaN
2,2013,00756,COMMONWEALTH CONNECTIONS ACADEMY CS,COMMONWEALTH CONNECTIONS ACADEMY CS,4050 CRUMS MILL ROAD,HARRISBURG,PA,17112,4050 CRUMS MILL RD,HARRISBURG,...,-76.8288,42043,8037,8768.0,8877.0,8974.0,8592.0,8761.0,9294.0,NaN
3,2013,01972,UPPER DARBY SHS,UPPER DARBY SD,601 NORTH LANSDOWNE AVENUE,DREXEL HILL,PA,19026,601 NORTH LANSDOWNE AVE,DREXEL HILL,...,-75.2849,42045,3638,3687.0,3625.0,3621.0,3717.0,3819.0,3845.0,NaN
4,2013,00026,HAZLETON AREA HS,HAZLETON AREA SD,1601 W 23RD ST,HAZLE TOWNSHIP,PA,18202,1601 WEST 23RD ST,HAZLE TOWNSHIP,...,-75.9958,42079,3207,3318.0,3304.0,3291.0,3425.0,3433.0,3513.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,2013,03832,TILDEN WILLIAM T MS,PHILADELPHIA CITY SD,6601 ELMWOOD AVE,PHILADELPHIA,PA,19142,6601 ELMWOOD AVE,PHILADELPHIA,...,-75.2339,42101,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3169,2013,03834,VAUX ROBERTS HS,PHILADELPHIA CITY SD,2300 W MASTER ST,PHILADELPHIA,PA,19121,2300 WEST MASTER ST,PHILADELPHIA,...,-75.1744,42101,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3170,2013,03840,BOK EDW W TECHNICAL HIGH SCHOOL,PHILADELPHIA CITY SD,1901 SOUTH 9TH STREET,PHILADELPHIA,PA,19148,1901 SOUTH 9TH ST,PHILADELPHIA,...,-75.1601,42101,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3171,2013,03852,GERMANTOWN HS,PHILADELPHIA CITY SD,40 E HIGH ST,PHILADELPHIA,PA,19144,40 EAST HIGH ST,PHILADELPHIA,...,-75.1766,42101,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
new_df['enrollment_change'] = new_df['enrollment19'] - new_df['enrollment']

In [37]:
fips_table = new_df.filter(['county_code','enrollment_change'], axis=1)
fips_table = fips_table.groupby('county_code',as_index=False).sum()
fips_table.head()

,county_code,enrollment_change
0,42001,-54.0
1,42003,-2426.0
2,42005,-880.0
3,42007,-1497.0
4,42009,-509.0


In [38]:
geo_df = gpd.read_file('georef.geojson')
geo_df = geo_df.filter(['coty_code','geometry'], axis=1)

In [39]:
geo_df.rename(columns={'coty_code':'county_code'}, inplace=True)

In [40]:
geo_df.head()

,county_code,geometry
0,29063,"POLYGON ((-94.60226 39.74719, -94.59837 39.747..."
1,29101,"POLYGON ((-94.06432 38.56738, -94.05580 38.567..."
2,29115,"POLYGON ((-93.36196 39.96761, -93.36136 39.967..."
3,29121,"POLYGON ((-92.69215 39.61026, -92.68930 39.610..."
4,29510,"POLYGON ((-90.18123 38.66007, -90.18260 38.665..."


In [41]:
merged = fips_table.merge(geo_df, on='county_code', how='left')

In [43]:
merged.head()

,county_code,enrollment_change,geometry
0,42001,-54.0,"POLYGON ((-76.99932 39.72011, -76.99932 39.722..."
1,42003,-2426.0,"POLYGON ((-80.14850 40.67377, -80.14838 40.670..."
2,42005,-880.0,"POLYGON ((-79.21521 41.05051, -79.21640 41.050..."
3,42007,-1497.0,"POLYGON ((-80.15853 40.85510, -80.16697 40.855..."
4,42009,-509.0,"POLYGON ((-78.13486 40.16513, -78.13472 40.165..."


In [50]:
import folium



SyntaxError: invalid syntax (<ipython-input-50-79536843c0b2>, line 2)

In [53]:

geo_list = merged['geometry'].tolist()

[<shapely.geometry.polygon.Polygon object at 0x7c20b2bc24c0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a519d0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a90c70>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a2d5e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa15b0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2bc24f0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa82e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa15e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a1bac0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a51130>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a5fa00>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a11eb0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a8a8e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a43730>, <shapely.geometry.multipolygon.MultiPolygon object at 0x7c20b2a11ee0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a43760>, <shapely.geom

In [56]:
m = folium.Map(location=[90, -45], zoom_start=3)

folium.Choropleth(
    geo_data=geo_list,
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Unemployment Rate (%)",
    reset=True,
).add_to(m)

m

ValueError: Cannot render objects with any missing geometries: [<shapely.geometry.polygon.Polygon object at 0x7c20b2bc24c0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a519d0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a90c70>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a2d5e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa15b0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2bc24f0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa82e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2aa15e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a1bac0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a51130>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a5fa00>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a11eb0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a8a8e0>, <shapely.geometry.polygon.Polygon object at 0x7c20b2a43730>, <shapely.geometry.multipolygon.MultiPolygon object at 0x7c20b2a11ee0>, <shapely.ge

SyntaxError: invalid syntax (<ipython-input-1-b7eb3c87a0b9>, line 1)